# 전처리 


In [2]:
import numpy as np
import pandas as pd

## 데이터 인코딩(Encoding)

### 데이터 타입 

데이터는 크게 2가지 형태로 나뉩니다.
- 수치형(Numerical)
- 범주형(Categorical)

> 📈 **Numerical (수치형) 데이터**
> - Discrete (이산형, 불연속형) 데이터  
     : 숫자로 셀 수 있는 것, 특정 값, 정수형태 (1개, 2명 등)
>- Continuous (연속형) 데이터  
     : 측정해야 하는 것, 연속 범위 내 실수값 (온도(21.3℃), 무게(72.5kg), 길이(183m), 비율(57%) 등)

> 📊 **Categorical (범주형) 데이터**
> - Nominal (범주형) 데이터  
     : 순서가 없는 것 (성별, 동물의 종류 등)  
> - Ordinal (순서형) 데이터  
     : 순서가 있는 것 (수능 등급, 좌석 등급, 순위 등)

하지만, 기본적인 머신러닝 모델은 categorical_data 를 입력값으로 활용할 수 없습니다.
따라서, categorical data 를 numerical 하게 변환해주는 전처리 작업(preprocessing)이 선행되어야합니다. 이러한 전처리작업을 인코딩 작업이라합니다. 

### 인코딩 방법 

- `sklearn` LabelEncoder 
- `pandas` pd.get_dummies
- `pandas` `map` 이용하기  

- pd.get_dummies
#### ③ **pandas**의 **get_dummies**  이용하기 🔩

위와 같은 인코딩(전처리) 방법 두 개는 간단하고 직관적이지만, 한 가지 문제가 있습니다 ⛔

> **[ 매우 싫다 -> 1, 보통이다 -> 3 , 매우 좋다 -> 5점 ]** 처럼  
ordianal한 데이터에는 적합하지만, (*순서 또는 등급 有)

> **[ 빨강 -> 1, 파랑 -> 2, 초록 -> 3 ]** 처럼  
nominal한 데이터에는 적합하지 않습니다. (*순서 또는 등급 無)

> 빨강, 파랑, 초록이라는 색깔들 사이에는 어떠한 순서나 등급도 없지만,  
위의 방법들로 변환하면 초록값이 가장 크다던지,  
"(초록값) - (파랑값) = (빨강값)" 등의 **관계**가 생기기 때문입니다.  
머신러닝 모델은 변환된 숫자를 바탕으로 학습하기 때문에, 잘못하면 위의 관계(특성)를 학습해 버립니다 😭😭  
이는 논리적으로 문제가 있을 뿐만 아니라, 실제 모델의 성능도 잘 안나오게 됩니다. 

**💡 따라서 nominal한 데이터 변환을 위해서는**, 숫자이기는 하지만 **서로 '관계없는' 숫자로 변환**해 주어야합니다.  

> - `pd.get_dummies(데이터셋.타겟열, prefix='타겟열')`

![예시](https://www.mwbrady.com/post/encodingcategoricaldata/Untitled-5f6d339d-ce4a-4533-8b67-c4e688b85ce0.png)

- map 

### LabelEncoder

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(
    {'pName':['냉장고','TV','에어컨','냉장고','노트북','공기청정기'],
     'price':[100,80,150,100,50,40]}
)

df

,pName,price
0,냉장고,100
1,TV,80
2,에어컨,150
3,냉장고,100
4,노트북,50
5,공기청정기,40


In [5]:
from sklearn.preprocessing import LabelEncoder # 함수 불러오기 
le = LabelEncoder() # 정의 
# fit_transform 라벨 인코딩 시행 
incode_name = le.fit_transform(df.pName)# 각 값들과 코드의 매핑 관계 생성 후 변환 

In [7]:
# pid 라는 열 만들고 값 넣기 
df['pid'] = incode_name
df

,pName,price,pid
0,냉장고,100,2
1,TV,80,0
2,에어컨,150,4
3,냉장고,100,2
4,노트북,50,3
5,공기청정기,40,1


In [8]:
print('Encoding Class : ', le.classes_)
print('Decoding Class: ', incode_name )

Encoding Class :  ['TV' '공기청정기' '냉장고' '노트북' '에어컨']
Decoding Class:  [2 0 4 2 3 1]


### OneHotEncoder

In [14]:
df

,pName,price,map_name
0,냉장고,100,2
1,TV,80,0
2,에어컨,150,4
3,냉장고,100,2
4,노트북,50,3
5,공기청정기,40,1


In [23]:
# 원핫인코딩 pd.get_dummies
df_dummies = pd.get_dummies(df)  # origin에 속한 카테고리(원산지)별로 새로운 comlumn(열)이 생김
df_dummies

,price,map_name,example,example_2,pName_TV,pName_공기청정기,pName_냉장고,pName_노트북,pName_에어컨
0,100,2,200,300,0,0,1,0,0
1,80,0,160,240,1,0,0,0,0
2,150,4,300,450,0,0,0,0,1
3,100,2,200,300,0,0,1,0,0
4,50,3,100,150,0,0,0,1,0
5,40,1,80,120,0,1,0,0,0


In [27]:
# pd.concat
pd.concat([df,df_dummies], axis=1)

,pName,price,pid,map_name,price,pid,pName_TV,pName_공기청정기,pName_냉장고,pName_노트북,pName_에어컨
0,냉장고,100,2,2,100,2,0,0,1,0,0
1,TV,80,0,0,80,0,1,0,0,0,0
2,에어컨,150,4,4,150,4,0,0,0,0,1
3,냉장고,100,2,2,100,2,0,0,1,0,0
4,노트북,50,3,3,50,3,0,0,0,1,0
5,공기청정기,40,1,1,40,1,0,1,0,0,0


### Map

In [10]:
# pid 컬럼 삭제 


In [11]:
df

,pName,price
0,냉장고,100
1,TV,80
2,에어컨,150
3,냉장고,100
4,노트북,50
5,공기청정기,40


In [12]:
# df = df.drop(['pid'], axis=1)
# pandas 에서 map 이란 ?
# 모든 각각의 원소를 ~  == map 
# {} = dict
# { key1: value1 , key2: value2, key3: value3}
df['map_name'] = df['pName'].map({'TV':0,'공기청정기':1,'냉장고':2,'노트북':3,'에어컨':4 })

In [20]:
df

,pName,price,map_name,example,example_2
0,냉장고,100,2,200,300
1,TV,80,0,160,240
2,에어컨,150,4,300,450
3,냉장고,100,2,200,300
4,노트북,50,3,100,150
5,공기청정기,40,1,80,120


<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i> 실제데이터에 인코딩해보기 
  </h2>
</div>

- Excercise 0: smoker,day, time 컬럼 드랍하여 tips 만들기 
- Excercise 1: total_bil의 범주형 데이터('sex') 를 수치형으로 바꾸기 
- Excercise 2: X,y 설정 ( y = total_bil // 나머지를 x 로 )
```python 
X = tips.drop(['total_bil'], axis=1)
y = tips.total_bil
```


<!-- - Excercise 3: x를 minmax scale 하기  -->
- Excercise 4: train_test_split 비율 0.3
- Excercise 5: LinearRegression 모델로 `total_bill` 예측해보기 
- Excercise 6: r2 스코어로 평가하기

In [79]:
import seaborn as sns # 그림 패키지
tips = sns.load_dataset('tips') # 내장 데이터 
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [80]:
tips.day.unique()

[Sun, Sat, Thur, Fri]
Categories (4, object): [Sun, Sat, Thur, Fri]

In [56]:
# Excercise 0: smoker,day, time 컬럼 드랍하여 tips 만들기
# sol1) 선택 
# tips = tips[['total_bill','tip','sex','size']]
# sol2) 드랍 
tips = tips.drop(['smoker','day','time'], axis=1)
tips

,total_bill,tip,sex,size
0,16.99,1.01,Female,2
1,10.34,1.66,Male,3
2,21.01,3.50,Male,3
3,23.68,3.31,Male,2
4,24.59,3.61,Female,4
...,...,...,...,...
239,29.03,5.92,Male,3
240,27.18,2.00,Female,2
241,22.67,2.00,Male,2
242,17.82,1.75,Male,2


In [51]:
# Excercise 1: total_bil의 범주형 데이터('sex') 를 수치형으로 바꾸기
# 1. map 
print(tips.sex.unique())
tips.sex = tips.sex.map({'Female':0, 'Male':1})
tips

In [52]:
# 2. sklearn.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #  정의 
decode = le.fit_transform(tips.sex) # 변환

# decode 값 추가 
tips['d_sex'] = decode

# 원래 컬럼 삭제 
tips = tips.drop(['sex'], axis=1)

In [60]:
# tips['sex'] == tips.sex
# tips['한 글'] == tips.한 글 <--- error
# 3. pd.get_dummies() : 원핫인코딩 
tmp = pd.get_dummies(tips['sex'])
tmp.head(3)

,Male,Female
0,0,1
1,1,0
2,1,0


In [62]:
# 삭제 
tips = tips.drop(['sex'],axis=1)
tips.head(1)

,total_bill,tip,size
0,16.99,1.01,2


In [64]:
# 붙이기 pd.concat([붙임1, 붙임2], axis=1)
# 붙임1과 붙임2를 합치는데 열로 합침 
tips = pd.concat([tips,tmp] ,axis=1)

In [65]:
tips

,total_bill,tip,size,Male,Female
0,16.99,1.01,2,0,1
1,10.34,1.66,3,1,0
2,21.01,3.50,3,1,0
3,23.68,3.31,2,1,0
4,24.59,3.61,4,0,1
...,...,...,...,...,...
239,29.03,5.92,3,1,0
240,27.18,2.00,2,0,1
241,22.67,2.00,2,1,0
242,17.82,1.75,2,1,0


In [67]:
# X, y 설정 
X = tips.drop('total_bill',axis=1)
y = tips['total_bill']

In [77]:
# Excercise 4: train_test_split 비율 0.3
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=42)

In [78]:
# Excercise 5: LinearRegression
from sklearn.linear_model import ElasticNet
model = ElasticNet() # 정의
model.fit(X_train, y_train) # 학습 

# Excercise 6: r2 스코어로 평가하기
model.score(X_test, y_test)

0.5257330656693666